In [33]:
import pyspark
from delta.pip_utils import configure_spark_with_delta_pip

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.0')  \
    .config("spark.hadoop.fs.s3a.endpoint", "localhost:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")

my_packages = ["org.apache.hadoop:hadoop-aws:3.3.1"]
spark = configure_spark_with_delta_pip(builder, extra_packages=my_packages).getOrCreate()

In [34]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [43]:
spark.read.format("delta").load("s3a://delta/silver/test11").createOrReplaceTempView("test")
spark.read.format("delta").load("s3a://delta/bronze/iot_poc_v18").createOrReplaceTempView("iot_poc_v1")


In [44]:
spark.sql("""\
SELECT beacon.received_timestamp, beacon.pub_key, witness.*
from test
where beacon.received_timestamp >= '2023-02-25 02:00:00' and 
      beacon.received_timestamp <= '2023-02-25 10:00:00' and
      selected = false and
      date = '2023-02-25'
limit 5
""").show(truncate=False)

+-----------------------+----------------------------------------------------+-----------------------+------------------+----------------------------------------------------+-----+-------+--------------------------------------------------------------------+-------------------+-----------------+----------+------------+----+--------+----------------------------------------------------------------------------------------------------+-----------+---------------------+----------------+--------+---------+
|received_timestamp     |pub_key                                             |received_timestamp     |location          |pub_key                                             |scale|status |data                                                                |timestamp_nanos    |concentrator_tmst|signal_dbm|frequency_hz|snr |datarate|signature                                                                                           |reward_unit|invalid_reason       |participant_side|gain_dbi|e